# Sample Towards Goal

In [ ]:
from numpy import arctan2
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import math

In [ ]:
%matplotlib inline

In [ ]:
seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

In [ ]:
x = np.array([0., 0., math.pi / 8.0, 0.0, 0.0])
goal = np.array([-.05, -.05])

In [ ]:
def towards_goal(curr_x):
    dt = 0.2
    mean_angle = (math.atan2(goal[1] - x[1], goal[0] - x[0]) - x[2]) / dt
    var_angle = 1.9/4
    # var_angle = 0.5
    angular_velocity = np.random.normal(mean_angle, var_angle**2)
    linear_velocity = 0.3
    return np.array([linear_velocity, angular_velocity])

In [ ]:
def motion(x: np.ndarray, u: np.ndarray) -> np.ndarray:
    """
    Describes how the robot moves
    :param x: current robot state
    :param u: action performed by the robot
    :return: the new robot state
    """
    dt = 0.2
    new_x = np.array(x, copy=True)
    # angle
    new_x[2] += u[1] * dt
    # vel lineare
    new_x[3] = u[0]
    # vel angolare
    new_x[4] = u[1]
    # x
    new_x[0] += u[0] * math.cos(new_x[2]) * dt
    # y
    new_x[1] += u[0] * math.sin(new_x[2]) * dt

    return new_x

In [ ]:
# ax.plot(sub_traj[:, 0], sub_traj[:, 1], "--r")

In [ ]:
actions = np.array([towards_goal(x) for _ in range(1000)])
states = np.array([motion(x=x, u=a) for a in actions])

In [ ]:
fig = plt.figure()
plt.cla()

plt.grid(True)
# plt.axis("equal")
vel_ang = (math.atan2(goal[1] - x[1], goal[0] - x[0]) - x[2]) / 0.2
mean = np.array([0.3, vel_ang])
x1 = motion(x=x, u=mean)

# for s in states:
#     fig.gca().plot([x[0], s[0]], [x[1], s[1]], "--g")

plt.gca().scatter(states[:, 0], states[:, 1])
# fig.gca().plot([x[0], x1[0]], [x[1], x1[1]], "--r")
# ROBOT POSITION
plt.plot(x[0], x[1], "xr")
# GOAL POSITION
plt.plot(goal[0], goal[1], "xb")
# plt.plot(10, 10, "xb")

fig.savefig("Sample_Towards.png", dpi=300)

# Action Evolution

In [ ]:
import seaborn as sns
import pandas as pd
sns.set_theme()

In [ ]:
actions = np.load("actions.npy")

In [ ]:
df = pd.DataFrame(
    {
        "Linear Velocity": actions[:, 0],
        "Angular Velocity": actions[:, 1]
    }
)
df = df.iloc[:500]

In [ ]:
len(df)

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
sns.lineplot(data=df,x=df.index, y="Linear Velocity", ax=axs[0], color= '#4c72b0')
sns.lineplot(data=df,x=df.index, y="Angular Velocity", ax=axs[1], color= '#c44e52')

# Tree Trajectories

## Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
%matplotlib inline

trajectories = np.load("trajectories.npz", allow_pickle=True)
values = np.load("rollout_values.npz", allow_pickle=True)
fig = plt.figure()
sim = trajectories['arr_0']
x = sim[0][0]

bottom_limit = -0.5
upper_limit = 11.5

right_limit = 11.5
left_limit = -0.5

ob = np.array([
        [4.5, 5.0],
        [5.0, 4.5],
        [5.0, 5.0],
        [5.0, 5.5],
        [5.5, 5.0],
    ])


for step_name in trajectories.files:
    step = trajectories[step_name]
    val_points = values[step_name]
    last_points = np.array([trj[-1][:2] for trj in step])
    x0 = step[0][0]
    
    fig = plt.figure()
    plt.cla()
    fig.gca().set_xlim([left_limit, right_limit])
    fig.gca().set_ylim([bottom_limit, upper_limit])
    plt.grid(True)
    # ROBOT POSITION
    plt.plot(x0[0], x0[1], "or")
    # OBSTACLES
    plt.plot(ob[:, 0], ob[:, 1], "ok")
    # GOAL 
    plt.plot(10, 10, "om")
    
    plt.scatter(last_points[:, 0], last_points[:, 1], c=val_points, marker='x')
    plt.colorbar()
    fig.savefig(f'trajectories_{step_name}.png', dpi=300)
    plt.close(fig)

## Animation

In [3]:
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

bottom_limit = -0.5
upper_limit = 11.5

right_limit = 11.5
left_limit = -0.5

ob = np.array([
        [4.5, 5.0],
        [5.0, 4.5],
        [5.0, 5.0],
        [5.0, 5.5],
        [5.5, 5.0],
    ])


def plot_frame(i, trajectories, values, fig):
    fig.clear()
    ax = fig.add_subplot()
    ax.set_title(f"Step: {i}")
    # plt.title(f"Step: {i}")
    file_name = trajectories.files[i]
    step = trajectories[file_name]
    val_points = values[file_name]
    
    last_points = np.array([trj[-1][:2] for trj in step])
    x0 = step[0][0]
    
    ax.cla()
    ax.set_xlim([left_limit, right_limit])
    ax.set_ylim([bottom_limit, upper_limit])
    ax.grid(True)
    # ROBOT POSITION
    ax.plot(x0[0], x0[1], "or")
    # OBSTACLES
    ax.plot(ob[:, 0], ob[:, 1], "ok")
    # GOAL 
    ax.plot(10, 10, "om")
    cmap = ax.scatter(last_points[:, 0], last_points[:, 1], c=val_points, marker='x')
    plt.colorbar(cmap)
    
trajectories = np.load("trajectories_0.npz", allow_pickle=True)
print(len(trajectories))
values = np.load("rollout_values_0.npz", allow_pickle=True)
print(len(values))
fig = plt.figure()

ani = FuncAnimation(
    fig,
    plot_frame,
    fargs=(trajectories, values, fig),
    frames=len(trajectories)
)
ani.save(f"animation.mp4", fps=5, dpi=300)
print("Done")

999
999
arr_369
arr_370
arr_371
arr_372
arr_373
arr_374
arr_375
arr_376
arr_377
arr_378
arr_379
arr_380
arr_381
arr_382
arr_383
arr_384
arr_385
arr_386
arr_387
arr_388
arr_389
arr_390
arr_391
arr_392
arr_393
arr_394
arr_395
arr_396
arr_397
arr_398
arr_399
arr_400
arr_401



KeyboardInterrupt



<Figure size 432x288 with 0 Axes>

# Heatmap Value

In [ ]:
from dataclasses import dataclass
from scipy.spatial.distance import cdist, euclidean
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import math
import seaborn as sns

In [ ]:
@dataclass
class Config:
    """
    simulation parameter class
    """
    # robot parameter
    # Max U[0]
    max_speed = 0.3  # [m/s]
    # Min U[0]
    min_speed = -0.1  # [m/s]
    # Max and Min U[1]
    # max_yaw_rate = 40.0 * math.pi / 180.0  # [rad/s]
    max_yaw_rate = 1.9  # [rad/s]

    # The action can be the relative change in speed -0.2 to 0.2
    # max_accel = 0.2  # [m/ss]
    # max_delta_yaw_rate = 40.0 * math.pi / 180.0  # [rad/ss]

    # yaw_rate_resolution = 0.1 * math.pi / 180.0  # [rad/s]
    dt = 0.2  # [s] Time tick for motion prediction
    robot_radius = 0.3  # [m] for collision check

    bottom_limit = -0.5
    upper_limit = 11.5

    right_limit = 11.5
    left_limit = -0.5
    obs_size = 0.2

    num_discrete_actions = 5

    # obstacles [x(m) y(m), ....]
    ob = np.array([
        [4.5, 5.0],
        [5.0, 4.5],
        [5.0, 5.0],
        [5.0, 5.5],
        [5.5, 5.0],
    ])

class RobotArenaState:
    def __init__(self, x: np.ndarray, goal: np.ndarray):
        # x, y, angle ,vel_lin, vel_ang
        self.x: np.ndarray = x
        # x(m), y(m)
        self.goal: np.ndarray = goal

def reward_grad(state: RobotArenaState, is_collision: bool, is_goal: bool,
                out_boundaries: bool) -> float:
    """
    Defines the reward the agent receives
    :param state: current robot state
    :param action: action performed by the agent
    :param is_collision: boolean value indicating if the robot is colliding
    :param is_goal: boolean value indicating if the robot has reached the goal
    :param out_boundaries: boolean value indicating if the robot is out of the map
    :return: The numerical reward of the agent
    """
    GOAL_REWARD: float = 1.0
    COLLISION_REWARD: float = -5.0

    if is_goal:
        return GOAL_REWARD

    if is_collision or out_boundaries:
        return COLLISION_REWARD
    max_eudist = 16.97056274847714
    
    # return -euclidean(state.x[:2], state.goal) / max_eudist
    return -math.hypot(state.goal[0]-state.x[0], state.goal[1]-state.x[1]) / max_eudist

def check_collision(x: np.ndarray) -> bool:
    """
    Check if the robot is colliding with some obstacle
    :param x: state of the robot
    :return:
    """
    config = Config()
    x = x[np.newaxis, ...]
    dist_to_obs: np.ndarray = cdist(x, config.ob, 'euclidean')
    return np.any(dist_to_obs <= config.robot_radius + config.obs_size)

def check_out_boundaries(state: RobotArenaState) -> bool:
    """
    Check if the agent goes out of the map
    :param state: state of the robot
    :return:
    """
    x_pos, y_pos = state.x
    c = Config()
    # Right and Left Map Limit
    if x_pos + c.robot_radius > c.right_limit or x_pos - c.robot_radius < c.left_limit:
        return True
    # Upper and Bottom Map Limit
    if y_pos + c.robot_radius > c.upper_limit or y_pos - c.robot_radius < c.bottom_limit:
        return True

    return False

def check_goal(state: RobotArenaState) -> bool:
    """
    Check if the robot reached the goal
    :param state: state of the robot
    :return:
    """
    x = state.x
    config = Config()
    goal = state.goal
    dist_to_goal = math.hypot(x[0] - goal[0], x[1] - goal[1])
    return dist_to_goal <= config.robot_radius

def get_rwrd(x_pos):
    state = RobotArenaState(
        x=x_pos,
        goal=np.array([10.0, 10.0])
    )
    collision = check_collision(state.x)
    goal = check_goal(state)
    out_boundaries = check_out_boundaries(state)
    return reward_grad(state, collision, goal, out_boundaries)

def motion(x: np.ndarray, u: np.ndarray) -> np.ndarray:
    """
    Describes how the robot moves
    :param x: current robot state
    :param u: action performed by the robot
    :return: the new robot state
    """
    dt = 0.2
    new_x = np.array(x, copy=True)
    # angle
    new_x[2] += u[1] * dt
    # vel lineare
    new_x[3] = u[0]
    # vel angolare
    new_x[4] = u[1]
    # x
    new_x[0] += u[0] * math.cos(new_x[2]) * dt
    # y
    new_x[1] += u[0] * math.sin(new_x[2]) * dt

    return new_x

config = Config()

In [ ]:
def random_action():
    low = [-0.1, -1.9]
    high = [0.3, 1.9]
    return np.array(
            [
                random.uniform(low[0], high[0]),
                random.uniform(low[1], high[1])
            ]
        )

In [ ]:
def move(x_pos):
    new_pos = x_pos
    tot = 0
    for i in range(5):
        a = random_action()
        print(a)
        print(new_pos)
        new_pos = motion(new_pos, a)
        r = get_rwrd(new_pos)
        tot += r * (0.99 ** i)
    return tot

In [ ]:
x = np.linspace(-0.5, 11.5, 1000)
y = np.linspace(-0.5, 11.5, 1000)
X, Y = np.meshgrid(x, y)
pos_matrix = np.stack([X, Y], axis=2)
zeros = np.zeros((len(x), len(y)))

In [ ]:
pos_matrix.shape

In [ ]:
for row_num in range(pos_matrix.shape[0]):
    for col_num in range(pos_matrix.shape[1]):
        zeros[row_num, col_num] = move(pos_matrix[row_num, col_num])
        # zeros[row_num, col_num] = get_rwrd(pos_matrix[row_num, col_num])

In [ ]:
fig, ax = plt.subplots()
cmap = ax.pcolormesh(X, Y, zeros, cmap="jet")
fig.colorbar(cmap)

In [ ]:
fig.savefig('heatmap_value.png', dpi=300)

# Heatmap

In [ ]:
from dataclasses import dataclass
from scipy.spatial.distance import cdist, euclidean
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import math
import seaborn as sns

In [ ]:
@dataclass
class Config:
    """
    simulation parameter class
    """
    # robot parameter
    # Max U[0]
    max_speed = 0.3  # [m/s]
    # Min U[0]
    min_speed = -0.1  # [m/s]
    # Max and Min U[1]
    # max_yaw_rate = 40.0 * math.pi / 180.0  # [rad/s]
    max_yaw_rate = 1.9  # [rad/s]

    # The action can be the relative change in speed -0.2 to 0.2
    # max_accel = 0.2  # [m/ss]
    # max_delta_yaw_rate = 40.0 * math.pi / 180.0  # [rad/ss]

    # yaw_rate_resolution = 0.1 * math.pi / 180.0  # [rad/s]
    dt = 0.2  # [s] Time tick for motion prediction
    robot_radius = 0.3  # [m] for collision check

    bottom_limit = -0.5
    upper_limit = 11.5

    right_limit = 11.5
    left_limit = -0.5
    obs_size = 0.2

    num_discrete_actions = 5

    # obstacles [x(m) y(m), ....]
    ob = np.array([
        [4.5, 5.0],
        [5.0, 4.5],
        [5.0, 5.0],
        [5.0, 5.5],
        [5.5, 5.0],
    ])

In [ ]:
class RobotArenaState:
    def __init__(self, x: np.ndarray, goal: np.ndarray):
        # x, y, angle ,vel_lin, vel_ang
        self.x: np.ndarray = x
        # x(m), y(m)
        self.goal: np.ndarray = goal

In [ ]:
def reward_grad(state: RobotArenaState, is_collision: bool, is_goal: bool,
                out_boundaries: bool) -> float:
    """
    Defines the reward the agent receives
    :param state: current robot state
    :param action: action performed by the agent
    :param is_collision: boolean value indicating if the robot is colliding
    :param is_goal: boolean value indicating if the robot has reached the goal
    :param out_boundaries: boolean value indicating if the robot is out of the map
    :return: The numerical reward of the agent
    """
    GOAL_REWARD: float = 1.0
    COLLISION_REWARD: float = -100.0

    if is_goal:
        return GOAL_REWARD

    if is_collision or out_boundaries:
        return COLLISION_REWARD
    max_eudist = 16.97056274847714
    
    # return -euclidean(state.x[:2], state.goal) / max_eudist
    return -math.hypot(state.goal[0]-state.x[0], state.goal[1]-state.x[1]) / max_eudist

In [ ]:
def check_collision(x: np.ndarray) -> bool:
    """
    Check if the robot is colliding with some obstacle
    :param x: state of the robot
    :return:
    """
    config = Config()
    x = x[np.newaxis, ...]
    dist_to_obs: np.ndarray = cdist(x, config.ob, 'euclidean')
    return np.any(dist_to_obs <= config.robot_radius + config.obs_size)

In [ ]:
def check_out_boundaries(state: RobotArenaState) -> bool:
    """
    Check if the agent goes out of the map
    :param state: state of the robot
    :return:
    """
    x_pos, y_pos = state.x
    c = Config()
    # Right and Left Map Limit
    if x_pos + c.robot_radius > c.right_limit or x_pos - c.robot_radius < c.left_limit:
        return True
    # Upper and Bottom Map Limit
    if y_pos + c.robot_radius > c.upper_limit or y_pos - c.robot_radius < c.bottom_limit:
        return True

    return False

In [ ]:
def check_goal(state: RobotArenaState) -> bool:
    """
    Check if the robot reached the goal
    :param state: state of the robot
    :return:
    """
    x = state.x
    config = Config()
    goal = state.goal
    dist_to_goal = math.hypot(x[0] - goal[0], x[1] - goal[1])
    return dist_to_goal <= config.robot_radius

In [ ]:
def get_rwrd(x_pos):
    state = RobotArenaState(
        x=x_pos,
        goal=np.array([10.0, 10.0])
    )
    collision = check_collision(state.x)
    goal = check_goal(state)
    out_boundaries = check_out_boundaries(state)
    return reward_grad(state, collision, goal, out_boundaries)

In [ ]:
config = Config()

In [ ]:
x = np.linspace(-0.5, 11.5, 1000)
y = np.linspace(-0.5, 11.5, 1000)

In [ ]:
X, Y = np.meshgrid(x, y)

In [ ]:
pos_matrix = np.stack([X, Y], axis=2)

In [ ]:
zeros = np.zeros((len(x), len(y)))

In [ ]:
for row_num in range(pos_matrix.shape[0]):
    for col_num in range(pos_matrix.shape[1]):
        zeros[row_num, col_num] = get_rwrd(pos_matrix[row_num, col_num])

In [ ]:
# np.apply_along_axis(get_rwrd, axis=2, arr=pos_matrix)

In [ ]:
fig, ax = plt.subplots()
cmap = ax.pcolormesh(X, Y, zeros, cmap="jet")
fig.colorbar(cmap)

In [ ]:
fig.savefig('heatmap.png', dpi=300)